In [1]:
import pandas as pd

# parquet 로드
df = pd.read_parquet("app_test_sample_id.parquet")

# 상위 5행 확인
display(df[df['sk_id_curr']==435478])


,sk_id_curr,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,...,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
94,435478,Cash loans,M,N,Y,1,202500.0,280170.0,29547.0,247500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# parquet 로드
model_df = pd.read_parquet("model_df.parquet")


# model_df.head()
# 상위 5행 확인

display(model_df[model_df['sk_id_curr']=='435478'])


,sk_id_curr,pd_hat,shap_features,shap_values,hcis_score,band,cutoff_score,margin_score,source_file
94,435478,0.201439,"[app_payment_rate, ext_source_3, years_beginex...","[-0.4736904501914978, -0.2673633396625519, 0.2...",699.353049,추가검토,675.0,24.35,app_test_sample_id.parquet


In [3]:
import pandas as pd

# =========================
# 1. 데이터 로드
# =========================
df = pd.read_csv("application_test.csv")

In [4]:
df.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
import pandas as pd
import numpy as np
import joblib
from pathlib import Path

from modules.preprocess import preprocess_features_only

pd.set_option("display.max_columns", 200)

# =========================
# 경로 설정
# =========================
DATA_DIR = Path("C:\\Users\\user\\Desktop\\streamlit\\st_data")
MODEL_DIR = Path("C:\\Users\\user\\Desktop\\streamlit\\artifacts\\model")

RAW_PATH = DATA_DIR / "application_test.csv"
MODEL_PATH = MODEL_DIR / "v1.0.1_XGB_artifact.joblib"   # 학습된 PD 모델
# OUT_PATH = DATA_DIR / "result.parquet"

# =========================
# Raw 데이터 로드
# =========================
df_raw = pd.read_csv(RAW_PATH)

# 컬럼명 소문자 통일
df_raw.columns = df_raw.columns.str.lower()

print("✅ Raw 데이터 로드 완료")

# =========================
# 전처리
# =========================
X, id_series = preprocess_features_only(df_raw)

print("✅ 전처리 완료")

✅ Raw 데이터 로드 완료
신청 데이터 셋 전체 ID 수 : 48744
✅ 전처리 완료


In [18]:
# 모델 번들 로드
bundle = joblib.load(MODEL_PATH)
model = bundle["model"]
FEATURES = bundle["feature_names"]
calibrator = bundle.get("calibrator", None)

print("X columns 예시:", list(X.columns)[:30])
print("모델 FEATURES 예시:", FEATURES[:30])

print("모델 기대 대비 누락:", len(set(FEATURES) - set(X.columns)))
print("X에만 있는 컬럼:", len(set(X.columns) - set(FEATURES)))

# 스키마 정렬/보정
def align_to_feature_schema(df: pd.DataFrame, feature_names: list) -> pd.DataFrame:
    df = df.copy()
    # 모델이 기대하는 컬럼이 없으면 0으로 생성
    for c in feature_names:
        if c not in df.columns:
            df[c] = 0
    # 모델이 기대하는 순서로 정렬 (여기서 불필요 컬럼 자동 제거)
    return df[feature_names]

df_model = align_to_feature_schema(X, FEATURES)

# 예측
pd_raw = model.predict_proba(df_model.values)[:, 1]

# calibrator 적용(있으면)
if calibrator is not None:
    if hasattr(calibrator, "predict_proba"):
        pd_hat = calibrator.predict_proba(pd_raw)
    elif hasattr(calibrator, "predict"):
        pd_hat = calibrator.predict(pd_raw)
    elif hasattr(calibrator, "transform"):
        pd_hat = calibrator.transform(pd_raw)
    else:
        raise AttributeError("calibrator에 predict/predict_proba/transform이 없습니다.")
else:
    pd_hat = pd_raw

pd_hat = np.clip(pd_hat, 1e-6, 1-1e-6)

X columns 예시: ['pre_app', 'bureau', 'bureau_bureau_bal', 'pos_cash', 'creditcard', 'installments', 'days_birth', 'days_id_publish', 'days_employed', 'days_last_phone_change', 'app_age_years', 'app_years_employed', 'app_employment_stability_ratio', 'amt_credit', 'amt_annuity', 'amt_goods_price', 'app_amt_credit_log', 'app_amt_annuity_log', 'app_amt_goods_price_log', 'app_annuity_income_ratio', 'app_payment_rate', 'ext_source_1', 'ext_source_2', 'ext_source_3', 'app_ext_source_min', 'app_ext_source_weighted', 'app_n_documents', 'flag_document_3', 'app_def_30_cnt_social_circle_clipped', 'own_car_age']
모델 FEATURES 예시: ['app_ext_source_min', 'ext_source_3', 'ext_source_2', 'app_payment_rate', 'app_ext_source_weighted', 'inst_delay_rate', 'ext_source_1', 'pre_annuity_mean', 'pre_credit_max', 'bu_any_over_limit_debt', 'own_car_age', 'pre_days_decision_mean', 'bu_total_debt_for_ratio', 'amt_goods_price', 'pre_credit_to_goods_mean', 'days_birth', 'app_years_employed', 'cc_util_mean', 'app_amt_g

In [ ]:
# =========================
# 점수 결과 테이블 구성 (ID + pd_hat)
# =========================
df_scored = df_model.copy()  # 모델 입력 스키마(=FEATURES 정렬된 DF)
df_scored["sk_id_curr"] = id_series.values  # preprocess_features_only가 준 id
df_scored["pd_hat"] = pd_hat

# =========================
# PD → HCIS
# =========================
def pd_to_hcis(pd_hat, offset=600, factor=50):
    odds = (1 - pd_hat) / pd_hat
    return offset + factor * np.log(odds)

df_scored["hcis_score"] = pd_to_hcis(df_scored["pd_hat"].values)

print("✅ HCIS 변환 완료")

# =========================
# HCIS Band
# =========================
T_LOW = 624
T_HIGH = 700

def hcis_band(score):
    if score < T_LOW:
        return "거절"
    elif score < T_HIGH:
        return "추가검토"
    return "승인"

df_scored["band"] = df_scored["hcis_score"].apply(hcis_band)

print("✅ Band 분류 완료")
print(df_scored["band"].value_counts(normalize=True))
print(df_scored["band"].value_counts())

# =========================
# 샘플링 (92:7:1, 100명)
# =========================
sample_plan = {"승인": 92, "추가검토": 7, "거절": 1}

samples = []
for band, n in sample_plan.items():
    df_band = df_scored[df_scored["band"] == band]
    if len(df_band) < n:
        raise ValueError(f"❌ {band} 샘플 부족: {len(df_band)} < {n}")
    samples.append(df_band.sample(n=n, random_state=42))

df_sample = (
    pd.concat(samples)
    .sample(frac=1, random_state=42)
    .reset_index(drop=True)
)

print("✅ 샘플링 완료")
print(df_sample["band"].value_counts())

✅ HCIS 변환 완료
✅ Band 분류 완료
band
추가검토    0.584154
거절      0.364455
승인      0.051391
Name: proportion, dtype: float64
band
추가검토    28474
거절      17765
승인       2505
Name: count, dtype: int64
✅ 샘플링 완료
band
승인      92
추가검토     7
거절       1
Name: count, dtype: int64


In [32]:
import pandas as pd

def show_duplicate_columns(df, name="df"):
    dup_mask = df.columns.duplicated(keep=False)
    dup_cols = df.columns[dup_mask]
    print(f"[{name}] n_cols={df.shape[1]}, duplicated_count={dup_mask.sum()}")
    if dup_mask.sum() > 0:
        print(f"[{name}] duplicated column names:")
        print(pd.Series(dup_cols).value_counts())

show_duplicate_columns(df_sample, "df_sample")
show_duplicate_columns(df_scored, "df_scored")


[df_sample] n_cols=114, duplicated_count=8
[df_sample] duplicated column names:
organization_type_nan    4
occupation_type_nan      4
Name: count, dtype: int64
[df_scored] n_cols=114, duplicated_count=8
[df_scored] duplicated column names:
organization_type_nan    4
occupation_type_nan      4
Name: count, dtype: int64


In [33]:
import pandas as pd
import numpy as np

def resolve_duplicate_columns(
    df: pd.DataFrame,
    verbose: bool = True
):
    """
    중복 컬럼 처리 규칙
    1) 값이 완전히 동일 → 하나만 남김
    2) 값이 다름 → suffix 붙여 모두 유지
    """
    df = df.copy()
    cols = pd.Series(df.columns)

    duplicates = cols[cols.duplicated()].unique()

    if verbose:
        print(f"🔎 중복 컬럼 후보 수: {len(duplicates)}")

    final_cols = []
    drop_cols = []
    rename_map = {}

    for col in duplicates:
        idxs = np.where(cols == col)[0]
        base_idx = idxs[0]
        base_series = df.iloc[:, base_idx]

        for i, idx in enumerate(idxs):
            cur_col = df.columns[idx]
            cur_series = df.iloc[:, idx]

            if i == 0:
                final_cols.append(cur_col)
                continue

            # 값 비교 (NaN 포함 동일성 체크)
            same = base_series.equals(cur_series)

            if same:
                drop_cols.append(cur_col)
                if verbose:
                    print(f"🟢 동일 → 제거: {col}")
            else:
                new_name = f"{col}__dup{i}"
                rename_map[cur_col] = new_name
                final_cols.append(new_name)
                if verbose:
                    print(f"🟡 다름 → 유지: {col} → {new_name}")

    # 실제 적용
    if drop_cols:
        df = df.drop(columns=drop_cols)

    if rename_map:
        df = df.rename(columns=rename_map)

    # 마지막 안전장치
    assert df.columns.is_unique, "❌ 아직도 중복 컬럼이 존재합니다!"

    if verbose:
        print(f"✅ 처리 완료 | 최종 컬럼 수: {df.shape[1]}")

    return df


,app_ext_source_min,ext_source_3,ext_source_2,app_payment_rate,app_ext_source_weighted,inst_delay_rate,ext_source_1,pre_annuity_mean,pre_credit_max,bu_any_over_limit_debt,own_car_age,pre_days_decision_mean,bu_total_debt_for_ratio,amt_goods_price,pre_credit_to_goods_mean,days_birth,app_years_employed,cc_util_mean,app_amt_goods_price_log,app_amt_annuity_log,pre_approval_ratio,amt_annuity,days_id_publish,app_n_documents,days_employed,inst_delay_days_mean,app_annuity_income_ratio,bu_ratio_active_loans,pre_credit_mean,cc_util_max,app_amt_credit_log,app_age_years,app_employment_stability_ratio,flag_document_3,app_def_30_cnt_social_circle_clipped,amt_credit,bu_days_credit_update_max,pre_approved_cnt,days_last_phone_change,pre_loan_duration_max,pre_new_cnt,region_rating_client_w_city,bu_cnt_active,pre_credit_min,pre_app,bureau,bureau_bureau_bal,pos_cash,creditcard,installments,code_gender_F,code_gender_M,code_gender_OTHER,code_gender_nan,name_education_type_Higher_education,name_education_type_Incomplete_higher,name_education_type_Lower_secondary,name_education_type_OTHER,name_education_type_Secondary_/_secondary_special,name_education_type_nan,name_family_status_Civil_marriage,name_family_status_Married,name_family_status_OTHER,name_family_status_Separated,name_family_status_Single_/_not_married,name_family_status_Widow,name_family_status_nan,organization_type_Business_Entity_Type_1,organization_type_Business_Entity_Type_2,organization_type_Business_Entity_Type_3,organization_type_Construction,organization_type_Government,organization_type_Industry__type_3,organization_type_Industry__type_9,organization_type_Kindergarten,organization_type_Medicine,organization_type_OTHER,organization_type_Other,organization_type_School,organization_type_Security,organization_type_Self-employed,organization_type_Trade__type_3,organization_type_Trade__type_7,organization_type_Transport__type_4,organization_type_nan,organization_type_nan,organization_type_nan,organization_type_nan,name_income_type_Commercial_associate,name_income_type_OTHER,name_income_type_Pensioner,name_income_type_State_servant,name_income_type_Working,name_income_type_nan,occupation_type_Accountants,occupation_type_Cleaning_staff,occupation_type_Cooking_staff,occupation_type_Core_staff,occupation_type_Drivers,occupation_type_High_skill_tech_staff,occupation_type_Laborers,occupation_type_Managers,occupation_type_Medicine_staff,occupation_type_OTHER,occupation_type_Sales_staff,occupation_type_Security_staff,occupation_type_nan,occupation_type_nan,occupation_type_nan,occupation_type_nan,sk_id_curr,pd_hat,hcis_score,band
0,0.781373,0.792264,0.781373,0.075347,0.517191,0.000000,0.506771,12057.075195,109264.5,0,0,-1096.000000,580014.000,180000.0,0.852927,-22687,4.835616,0.135683,12.100718,9.728300,0.852927,16785.0,-4391,1,0,3.819444,0.162174,0.500000,80016.750000,0.690412,12.313891,62.2,0.128198,1,0.0,222768.0,-21,1.000000,-1583,270,0.50,2,2.0,50769.0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,203124,0.112820,703.112932,승인
1,0.520267,0.656158,0.520267,0.044036,0.716057,0.000000,0.857489,10747.958008,405000.0,0,0,-520.857143,176095.215,126000.0,1.012356,-20904,4.835616,0.000000,11.744045,8.621463,1.012356,5548.5,-4266,1,0,3.819444,0.035229,0.375000,104610.210938,0.000000,11.744045,57.3,0.128198,1,0.0,126000.0,-14,0.700000,-310,510,0.00,3,6.0,0.0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,396922,0.065726,732.713710,승인
2,0.632978,0.519097,0.632978,0.102460,0.517191,0.027778,0.506771,10258.132812,287685.0,0,0,-1791.000000,0.000,225000.0,1.068289,-19163,18.320548,0.135683,12.323860,10.045616,1.068289,23053.5,-2719,1,-6687,1.000000,0.256150,0.000000,177079.500000,0.690412,12.323860,52.5,0.348963,1,0.0,225000.0,-2539,1.000000,-2422,1050,0.00,2,0.0,66474.0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [34]:
sample_safe = df_sample.copy()
scored_safe = df_scored.copy()

In [47]:
sample_safe[sample_safe['band']=='추가검토']

,app_ext_source_min,ext_source_3,ext_source_2,app_payment_rate,app_ext_source_weighted,inst_delay_rate,ext_source_1,pre_annuity_mean,pre_credit_max,bu_any_over_limit_debt,own_car_age,pre_days_decision_mean,bu_total_debt_for_ratio,amt_goods_price,pre_credit_to_goods_mean,days_birth,app_years_employed,cc_util_mean,app_amt_goods_price_log,app_amt_annuity_log,pre_approval_ratio,amt_annuity,days_id_publish,app_n_documents,days_employed,inst_delay_days_mean,app_annuity_income_ratio,bu_ratio_active_loans,pre_credit_mean,cc_util_max,app_amt_credit_log,app_age_years,app_employment_stability_ratio,flag_document_3,app_def_30_cnt_social_circle_clipped,amt_credit,bu_days_credit_update_max,pre_approved_cnt,days_last_phone_change,pre_loan_duration_max,pre_new_cnt,region_rating_client_w_city,bu_cnt_active,pre_credit_min,pre_app,bureau,bureau_bureau_bal,pos_cash,creditcard,installments,code_gender_F,code_gender_M,code_gender_OTHER,code_gender_nan,name_education_type_Higher_education,name_education_type_Incomplete_higher,name_education_type_Lower_secondary,name_education_type_OTHER,name_education_type_Secondary_/_secondary_special,name_education_type_nan,name_family_status_Civil_marriage,name_family_status_Married,name_family_status_OTHER,name_family_status_Separated,name_family_status_Single_/_not_married,name_family_status_Widow,name_family_status_nan,organization_type_Business_Entity_Type_1,organization_type_Business_Entity_Type_2,organization_type_Business_Entity_Type_3,organization_type_Construction,organization_type_Government,organization_type_Industry__type_3,organization_type_Industry__type_9,organization_type_Kindergarten,organization_type_Medicine,organization_type_OTHER,organization_type_Other,organization_type_School,organization_type_Security,organization_type_Self-employed,organization_type_Trade__type_3,organization_type_Trade__type_7,organization_type_Transport__type_4,organization_type_nan,organization_type_nan,organization_type_nan,organization_type_nan,name_income_type_Commercial_associate,name_income_type_OTHER,name_income_type_Pensioner,name_income_type_State_servant,name_income_type_Working,name_income_type_nan,occupation_type_Accountants,occupation_type_Cleaning_staff,occupation_type_Cooking_staff,occupation_type_Core_staff,occupation_type_Drivers,occupation_type_High_skill_tech_staff,occupation_type_Laborers,occupation_type_Managers,occupation_type_Medicine_staff,occupation_type_OTHER,occupation_type_Sales_staff,occupation_type_Security_staff,occupation_type_nan,occupation_type_nan,occupation_type_nan,occupation_type_nan,sk_id_curr,pd_hat,hcis_score,band
27,0.371360,0.495666,0.371360,0.051050,0.547823,0.111111,0.674564,8679.041992,208053.0,1,0,-1062.200000,1711566.0,270000.0,1.047579,-10630,3.394521,0.135683,12.506181,9.531300,1.047579,13783.5,-2511,1,-1239,2.666667,0.109393,0.400000,94006.796875,0.690412,12.506181,29.1,0.116650,1,0.0,270000.0,-4,0.8,-220,480,0.200000,3,8.0,19683.0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,408986,0.410526,618.089502,추가검토
34,0.469091,0.519097,0.564312,0.097519,0.517191,0.000000,0.469091,29568.087891,900000.0,0,20,-578.666667,14341.5,1309500.0,1.125752,-16680,4.835616,0.000000,14.085157,11.724753,1.125752,123592.5,-213,1,0,3.819444,0.436585,0.500000,511078.500000,0.000000,14.085157,45.7,0.128198,1,0.0,1309500.0,-46,1.0,-722,60,0.000000,2,1.0,64426.5,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,207339,0.288709,645.083141,추가검토
43,0.146960,0.593718,0.663051,0.079258,0.391139,0.000000,0.146960,22115.789062,178960.5,0,0,-248.000000,0.0,450000.0,0.899995,-8925,1.460274,0.135683,13.017005,10.605966,0.899995,40374.0,-1585,1,-533,3.819444,0.179440,1.000000,178960.500000,0.690412,13.140991,24.5,0.059603,0,1.0,509400.0,-220,1.0,-248,-27766,1.000000,1,1.0,178960.5,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [35]:
df_sample_clean = resolve_duplicate_columns(df_sample)
df_scored_clean = resolve_duplicate_columns(df_scored)

df_sample_clean.to_parquet(DATA_DIR / "sample.parquet", index=False)
df_scored_clean.to_parquet(DATA_DIR / "scored.parquet", index=False)

print("✅ parquet 저장 성공")


🔎 중복 컬럼 후보 수: 2
🟢 동일 → 제거: organization_type_nan
🟢 동일 → 제거: organization_type_nan
🟢 동일 → 제거: organization_type_nan
🟢 동일 → 제거: occupation_type_nan
🟢 동일 → 제거: occupation_type_nan
🟢 동일 → 제거: occupation_type_nan
✅ 처리 완료 | 최종 컬럼 수: 106
🔎 중복 컬럼 후보 수: 2
🟢 동일 → 제거: organization_type_nan
🟢 동일 → 제거: organization_type_nan
🟢 동일 → 제거: organization_type_nan
🟢 동일 → 제거: occupation_type_nan
🟢 동일 → 제거: occupation_type_nan
🟢 동일 → 제거: occupation_type_nan
✅ 처리 완료 | 최종 컬럼 수: 106
✅ parquet 저장 성공


In [41]:
df.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0505,NaN,NaN,0.0672,0.0612,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0526,NaN,NaN,0.0666,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0514,NaN,NaN,NaN,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 3,NaN,0.699787,0.610991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,Working,Secondary / secondary special,Married

In [42]:
app_test_sample_id = df_sample['sk_id_curr']

app_test_filtered = df_raw[
    df_raw['sk_id_curr'].isin(app_test_sample_id)
].copy()


In [43]:
app_test_filtered.to_parquet(DATA_DIR / 'app_test_sample_id.parquet', index=False)

In [48]:
import pandas as pd
from pathlib import Path

# 같은 폴더 경로
DATA_DIR = Path(".")

# 파일 로드
app_test = pd.read_parquet(DATA_DIR / "app_test_4.parquet")
app_test_sample_id = pd.read_parquet(DATA_DIR / "app_test_sample_id.parquet")

print("✅ 파일 로드 완료")


✅ 파일 로드 완료


In [50]:
app_test_sample_id = pd.read_parquet(DATA_DIR / "app_test_sample_id.parquet")

print("shape:", app_test_sample_id.shape)

shape: (100, 121)


In [45]:
print("=== app_test ===")
print("shape:", app_test.shape)
print("columns:", app_test.columns.tolist())
display(app_test.head())

print("\n=== app_test_sample_id ===")
print("shape:", app_test_sample_id.shape)
print("columns:", app_test_sample_id.columns.tolist())
display(app_test_sample_id.head())

=== app_test ===
shape: (4, 121)
columns: ['sk_id_curr', 'name_contract_type', 'code_gender', 'flag_own_car', 'flag_own_realty', 'cnt_children', 'amt_income_total', 'amt_credit', 'amt_annuity', 'amt_goods_price', 'name_type_suite', 'name_income_type', 'name_education_type', 'name_family_status', 'name_housing_type', 'region_population_relative', 'days_birth', 'days_employed', 'days_registration', 'days_id_publish', 'own_car_age', 'flag_mobil', 'flag_emp_phone', 'flag_work_phone', 'flag_cont_mobile', 'flag_phone', 'flag_email', 'occupation_type', 'cnt_fam_members', 'region_rating_client', 'region_rating_client_w_city', 'weekday_appr_process_start', 'hour_appr_process_start', 'reg_region_not_live_region', 'reg_region_not_work_region', 'live_region_not_work_region', 'reg_city_not_live_city', 'reg_city_not_work_city', 'live_city_not_work_city', 'organization_type', 'ext_source_1', 'ext_source_2', 'ext_source_3', 'apartments_avg', 'basementarea_avg', 'years_beginexpluatation_avg', 'years_bu

,sk_id_curr,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,name_type_suite,name_income_type,name_education_type,name_family_status,name_housing_type,region_population_relative,days_birth,days_employed,days_registration,days_id_publish,own_car_age,flag_mobil,flag_emp_phone,flag_work_phone,flag_cont_mobile,flag_phone,flag_email,occupation_type,cnt_fam_members,region_rating_client,region_rating_client_w_city,weekday_appr_process_start,hour_appr_process_start,reg_region_not_live_region,reg_region_not_work_region,live_region_not_work_region,reg_city_not_live_city,reg_city_not_work_city,live_city_not_work_city,organization_type,ext_source_1,ext_source_2,ext_source_3,apartments_avg,basementarea_avg,years_beginexpluatation_avg,years_build_avg,commonarea_avg,elevators_avg,entrances_avg,floorsmax_avg,floorsmin_avg,landarea_avg,livingapartments_avg,livingarea_avg,nonlivingapartments_avg,nonlivingarea_avg,apartments_mode,basementarea_mode,years_beginexpluatation_mode,years_build_mode,commonarea_mode,elevators_mode,entrances_mode,floorsmax_mode,floorsmin_mode,landarea_mode,livingapartments_mode,livingarea_mode,nonlivingapartments_mode,nonlivingarea_mode,apartments_medi,basementarea_medi,years_beginexpluatation_medi,years_build_medi,commonarea_medi,elevators_medi,entrances_medi,floorsmax_medi,floorsmin_medi,landarea_medi,livingapartments_medi,livingarea_medi,nonlivingapartments_medi,nonlivingarea_medi,fondkapremont_mode,housetype_mode,totalarea_mode,wallsmaterial_mode,emergencystate_mode,obs_30_cnt_social_circle,def_30_cnt_social_circle,obs_60_cnt_social_circle,def_60_cnt_social_circle,days_last_phone_change,flag_document_2,flag_document_3,flag_document_4,flag_document_5,flag_document_6,flag_document_7,flag_document_8,flag_document_9,flag_document_10,flag_document_11,flag_document_12,flag_document_13,flag_document_14,flag_document_15,flag_document_16,flag_document_17,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
6017,144092,Cash loans,M,N,Y,0,256500.0,609898.5,31270.5,526500.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.002042,-15214,-2773,-2075.0,-4943,NaN,1,1,0,1,0,1,Laborers,2.0,3,3,FRIDAY,10,0,0,0,0,0,0,Other,0.460429,0.771297,0.410103,0.0557,0.0000,0.9861,0.8096,0.0000,0.080,0.0690,0.3333,0.375,0.0672,0.0454,0.0591,0.0000,0.0385,0.0567,0.0000,0.9861,0.8171,0.0000,0.0806,0.0690,0.3333,0.375,0.0687,0.0496,0.0616,0.0000,0.0408,0.0562,0.0000,0.9861,0.8121,0.0000,0.08,0.0690,0.3333,0.375,0.0684,0.0462,0.0602,0.0000,0.0393,None,block of flats,0.0489,Panel,No,0.0,0.0,0.0,0.0,-790.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0
10157,173779,Cash loans,F,N,Y,0,67500.0,296280.0,19062.0,225000.0,Unaccompanied,Pensioner,Higher education,Single / not married,House / apartment,0.009175,-19852,365243,-192.0,-3237,NaN,1,0,0,1,0,0,None,1.0,2,2,FRIDAY,14,0,0,0,0,0,0,XNA,NaN,0.672038,0.554947,0.2320,0.1576,0.9990,0.9864,0.0803,0.200,0.1724,0.3333,0.375,0.0291,0.1891,0.2099,0.8687,0.0000,0.2363,0.1635,0.9990,0.9869,0.0810,0.2014,0.1724,0.3333,0.375,0.0297,0.2066,0.2186,0.8755,0.0000,0.2342,0.1576,0.9990,0.9866,0.0808,0.20,0.1724,0.3333,0.375,0.0296,0.1924,0.2136,0.8734,0.0000,reg oper spec account,block of flats,0.2187,"Stone, brick",No,0.0,0.0,0.0,0.0,-693.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
15071,208550,Cash loans,M,Y,Y,2,450000.0,854896.5,36351.0,702000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.032561,-11348,-1149,-5443.0,-3953,14.0,1,1,0,1,0,0,Managers,4.0,1,1,MONDAY,13,0,0,0,0,0,0,Business Entity Type 3,0.637061,0.771890,NaN,0.0926,0.0568,0.9732,0.6328,0.0115,0.072,0.1379,0.1333,0.175,0.0379,0.0751,0.0898,0.0015,0.0081,0.0378,0.0413,0.9727,0.6406,0.0026,0.0000,0.1034,0.0833,0.125,0.0


=== app_test_sample_id ===
shape: (100, 121)
columns: ['sk_id_curr', 'name_contract_type', 'code_gender', 'flag_own_car', 'flag_own_realty', 'cnt_children', 'amt_income_total', 'amt_credit', 'amt_annuity', 'amt_goods_price', 'name_type_suite', 'name_income_type', 'name_education_type', 'name_family_status', 'name_housing_type', 'region_population_relative', 'days_birth', 'days_employed', 'days_registration', 'days_id_publish', 'own_car_age', 'flag_mobil', 'flag_emp_phone', 'flag_work_phone', 'flag_cont_mobile', 'flag_phone', 'flag_email', 'occupation_type', 'cnt_fam_members', 'region_rating_client', 'region_rating_client_w_city', 'weekday_appr_process_start', 'hour_appr_process_start', 'reg_region_not_live_region', 'reg_region_not_work_region', 'live_region_not_work_region', 'reg_city_not_live_city', 'reg_city_not_work_city', 'live_city_not_work_city', 'organization_type', 'ext_source_1', 'ext_source_2', 'ext_source_3', 'apartments_avg', 'basementarea_avg', 'years_beginexpluatation_av

,sk_id_curr,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,name_type_suite,name_income_type,name_education_type,name_family_status,name_housing_type,region_population_relative,days_birth,days_employed,days_registration,days_id_publish,own_car_age,flag_mobil,flag_emp_phone,flag_work_phone,flag_cont_mobile,flag_phone,flag_email,occupation_type,cnt_fam_members,region_rating_client,region_rating_client_w_city,weekday_appr_process_start,hour_appr_process_start,reg_region_not_live_region,reg_region_not_work_region,live_region_not_work_region,reg_city_not_live_city,reg_city_not_work_city,live_city_not_work_city,organization_type,ext_source_1,ext_source_2,ext_source_3,apartments_avg,basementarea_avg,years_beginexpluatation_avg,years_build_avg,commonarea_avg,elevators_avg,entrances_avg,floorsmax_avg,floorsmin_avg,landarea_avg,livingapartments_avg,livingarea_avg,nonlivingapartments_avg,nonlivingarea_avg,apartments_mode,basementarea_mode,years_beginexpluatation_mode,years_build_mode,commonarea_mode,elevators_mode,entrances_mode,floorsmax_mode,floorsmin_mode,landarea_mode,livingapartments_mode,livingarea_mode,nonlivingapartments_mode,nonlivingarea_mode,apartments_medi,basementarea_medi,years_beginexpluatation_medi,years_build_medi,commonarea_medi,elevators_medi,entrances_medi,floorsmax_medi,floorsmin_medi,landarea_medi,livingapartments_medi,livingarea_medi,nonlivingapartments_medi,nonlivingarea_medi,fondkapremont_mode,housetype_mode,totalarea_mode,wallsmaterial_mode,emergencystate_mode,obs_30_cnt_social_circle,def_30_cnt_social_circle,obs_60_cnt_social_circle,def_60_cnt_social_circle,days_last_phone_change,flag_document_2,flag_document_3,flag_document_4,flag_document_5,flag_document_6,flag_document_7,flag_document_8,flag_document_9,flag_document_10,flag_document_11,flag_document_12,flag_document_13,flag_document_14,flag_document_15,flag_document_16,flag_document_17,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
0,110258,Cash loans,F,N,N,0,67500.0,112500.0,9652.5,112500.0,Family,Working,Secondary / secondary special,Separated,House / apartment,0.010276,-18827,-7707,-8223.0,-2359,NaN,1,1,1,1,0,0,Laborers,1.0,2,2,FRIDAY,10,0,0,0,0,0,0,Industry: type 3,NaN,0.714150,0.749022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,None,None,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0
1,114653,Cash loans,M,Y,N,0,247500.0,233833.5,24565.5,211500.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.025164,-14919,-8126,-4934.0,-5368,3.0,1,1,0,1,0,0,High skill tech staff,2.0,2,2,SATURDAY,12,0,0,0,1,0,1,Industry: type 9,NaN,0.622354,0.801601,0.0165,0.0000,0.9722,0.6192,0.0013,0.00,0.0690,0.0417,0.0833,0.0196,0.0134,0.0124,0.000,0.0000,0.0168,0.0000,0.9722,0.6341,0.0014,0.0000,0.0690,0.0417,0.0833,0.0200,0.0147,0.0129,0.0000,0.0000,0.0167,0.0000,0.9722,0.6243,0.0014,0.00,0.0690,0.0417,0.0833,0.0199,0.0137,0.0126,0.0000,0.0000,reg oper spec account,block of flats,0.0098,"Stone, brick",No,0.0,0.0,0.0,0.0,-910.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
2,119718,Cash loans,F,N,Y,2,180000.0,162000.0,16150.5,162000.0,Unaccompanied,Commercial associate,Higher education,Civil marriage,With parents,0.046220,-15786,-2474,-1603.0,-5059,NaN,1,1,0,1,0,0,Core staff,4.0,1,1,THURSDAY,13,0,0,0,0,0,0,Other,0.487155,0.738847,0.627991,0.2227,0.1649,0.9791,0.7144,0.0619,0.24,0.2069,0.3333,0.3750,0.2365,0.1816,0.2253,0.000,0.0000,0.2269,0.1712,0.9791,0.7256,0.0625,0.2417,0.2069,0.3333,0.3750,0.2419,0.1983,0.2348,0.0000,0.0000,0.2248,0.1649,0.9791,0.7182,0.0623,0.24,0.2069,0.3333,0.3750,0.2406,0.1847,0.2294,0.0000,0

In [26]:
print("len(FEATURES):", len(FEATURES))
print("X shape:", X.shape)
print("X head cols:", X.columns[:50].tolist())
print("pre_ cols:", [c for c in X.columns if c.startswith("pre_")][:20])
print("bu_ cols:", [c for c in X.columns if c.startswith("bu_")][:20])
print("cc_ cols:", [c for c in X.columns if c.startswith("cc_")][:20])
print("inst_ cols:", [c for c in X.columns if c.startswith("inst_")][:20])


len(FEATURES): 110
X shape: (48744, 56)
X head cols: ['pre_app', 'bureau', 'bureau_bureau_bal', 'pos_cash', 'creditcard', 'installments', 'days_birth', 'days_id_publish', 'days_employed', 'days_last_phone_change', 'app_age_years', 'app_years_employed', 'app_employment_stability_ratio', 'amt_credit', 'amt_annuity', 'amt_goods_price', 'app_amt_credit_log', 'app_amt_annuity_log', 'app_amt_goods_price_log', 'app_annuity_income_ratio', 'app_payment_rate', 'ext_source_1', 'ext_source_2', 'ext_source_3', 'app_ext_source_min', 'app_ext_source_weighted', 'app_n_documents', 'flag_document_3', 'app_def_30_cnt_social_circle_clipped', 'own_car_age', 'code_gender', 'name_family_status', 'region_rating_client_w_city', 'organization_type', 'name_income_type', 'occupation_type', 'name_education_type', 'pre_annuity_mean', 'pre_credit_max', 'pre_days_decision_mean', 'pre_credit_to_goods_mean', 'pre_approval_ratio', 'pre_credit_mean', 'pre_approved_cnt', 'pre_new_cnt', 'pre_loan_duration_max', 'pre_credit

In [30]:
# =========================
# 저장
# =========================
df_sample.to_parquet(DATA_DIR / 'sample.parquet', index=False)

print(f"✅ 저장 완료 → {DATA_DIR}")


ValueError: Duplicate column names found: ['app_ext_source_min', 'ext_source_3', 'ext_source_2', 'app_payment_rate', 'app_ext_source_weighted', 'inst_delay_rate', 'ext_source_1', 'pre_annuity_mean', 'pre_credit_max', 'bu_any_over_limit_debt', 'own_car_age', 'pre_days_decision_mean', 'bu_total_debt_for_ratio', 'amt_goods_price', 'pre_credit_to_goods_mean', 'days_birth', 'app_years_employed', 'cc_util_mean', 'app_amt_goods_price_log', 'app_amt_annuity_log', 'pre_approval_ratio', 'amt_annuity', 'days_id_publish', 'app_n_documents', 'days_employed', 'inst_delay_days_mean', 'app_annuity_income_ratio', 'bu_ratio_active_loans', 'pre_credit_mean', 'cc_util_max', 'app_amt_credit_log', 'app_age_years', 'app_employment_stability_ratio', 'flag_document_3', 'app_def_30_cnt_social_circle_clipped', 'amt_credit', 'bu_days_credit_update_max', 'pre_approved_cnt', 'days_last_phone_change', 'pre_loan_duration_max', 'pre_new_cnt', 'region_rating_client_w_city', 'bu_cnt_active', 'pre_credit_min', 'pre_app',

In [3]:
import xgboost
xgboost.__version__


'3.1.2'